---
### Chargmenet et Verification des images
Charger les images du dataset et vérifier leurs extensions (jpeg, jpg, bmp, png). Supprimer celles qui ne correspondent pas.


In [ ]:
import os 

# data path
data_path = "data/data_blood_cell_cancer/" 


folders = os.listdir(data_path)

invalid_files  = []
classesWithoutExt  = {}

filescounter =0

allowed_extensions = [".jpeg", ".jpg", ".bmp", ".png"]

for folder in folders:
    folder_path = os.path.join(data_path, folder)

    # Skip if not a directory
    if not os.path.isdir(folder_path):
        continue

    classesWithoutExt[folder]=0

    for file in os.listdir(folder_path):

        file_path = os.path.join(folder_path, file)
        name,ext = os.path.splitext(file)

        filescounter+=1

        try:
            if ext.lower() == "" or ext.lower() not in allowed_extensions:
                classesWithoutExt[folder]+=1
                invalid_files.append(file)
                
                print(f"Deleting file without extension: {file_path}")

                # data/folder/image
                # os.remove(file_path)
        except FileNotFoundError:
            print(f"Fichier introuvable : {file_path}")
        except Exception as e:
            print(f"Erreur inattendue avec {file_path}: {e}")



print("total fils: ",filescounter)
print("files withiut extention ",len(invalid_files)) # 32 file witout extention
print(classesWithoutExt) 

# output
# 32
# {'glioma': 6, 'meningioma': 10, 'notumor': 7, 'pituitary': 9}


total fils:  3242
files withiut extention  0
{'Benign': 0, 'early Pre-B': 0, 'Pre-B': 0, 'Pro-B': 0}
